# Step #1 Setup 

In [2]:
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient
import openai
import logging
import tiktoken
import pandas as pd
import pymongo
from dotenv import load_dotenv
load_dotenv()

# Set up the Azure Key Vault client and retrieve the Blob Storage account credentials
keyvault_name = 'ffkeyvault731'
client = SecretClient(f"https://{keyvault_name}.vault.azure.net/", AzureCliCredential())
print('keyvault service ready')

# AzureOpenAI Service
def setup_azureopenai():
    openai.api_key = client.get_secret('openai-api-key').value
    openai.api_type = "azure"
    openai.api_base = 'https://openai713.openai.azure.com'
    openai.api_version = '2023-05-15'
    print('azure openai service ready')

# public openai service
def setup_public_openai():
    openai.api_key = client.get_secret('openai-api-key-public').value
    print('public openai service ready')

DB_NAME = "hephaestus"
COLLECTION_NAME = 'isocodes'

def setup_cosmos_connection():
    COSMOS_CLUSTER_CONNECTION_STRING = client.get_secret('cosmos-cluster-string').value
    cosmosclient = pymongo.MongoClient(COSMOS_CLUSTER_CONNECTION_STRING)

    db = cosmosclient[DB_NAME]
    collection = cosmosclient[DB_NAME][COLLECTION_NAME]

    # Send a ping to confirm a successful connection
    try:
        cosmosclient.admin.command('ping')
        print("Pinged your deployment. You successfully connected to MongoDB!")
    except Exception as e:
        print(e)
    return collection, db

setup_public_openai()
collection, db = setup_cosmos_connection()

keyvault service ready
public openai service ready
Pinged your deployment. You successfully connected to MongoDB!


# Step #2 Populating the Vector DB

In [3]:
# prepare content for insertion into cosmos db
def prepare_content(text_content):
  embeddings = create_embeddings_with_openai(text_content)
  request = [
    {
    "textContent": text_content, 
    "vectorContent": embeddings}
  ]
  return request


# create embeddings
def create_embeddings_with_openai(input):
    print('Generating response from OpenAI...')

    ###### uncomment for AzureOpenAI model usage and comment code below

    # embeddings = openai.Embedding.create( 
    #     engine='deployment-9521a59a710e46d7b99a60700d6ae119', 
    #     input=input)["data"][0]["embedding"]

    embeddings = openai.Embedding.create(
        model='text-embedding-ada-002', 
        input=input)["data"][0]["embedding"]
    
    # Number of embeddings    
    print(len(embeddings))

    return embeddings

# Resets the DB and deletes all values from the collection to avoid dublicates
#collection.delete_many({})

# insert the requests
def insert_requests(text_input):
    request = prepare_content(text_input)
    return collection.insert_many(request).inserted_ids


In [4]:
# # insert some sample data for testing purposes
# content_list = ['This is a text on aliens',
#                 'This is another text on aliens', 
#                 'random text', 
#                 'openai works great']

# for text_input in content_list:
#     insert_requests(text_input)

# # print the number of documents in the collection
# print(db.isocodes.count_documents({}))

In [11]:
# document cracking function to insert data from the excel sheet

import PyPDF2
import re

def split_text_into_paragraphs(text):
    paragraphs = re.split(r'\n{2,}', text)
    return paragraphs

def slice_pdf_into_records(pdf_path, max_sentences):
    records = []
    
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        for page in reader.pages:
            text = page.extract_text()
            paragraphs = split_text_into_paragraphs(text)
            
            current_record = ''
            sentence_count = 0
            
            for paragraph in paragraphs:
                sentences = re.split(r'(?<=[.!?])\s+', paragraph)
                
                for sentence in sentences:
                    current_record += sentence
                    
                    sentence_count += 1
                    
                    if sentence_count >= max_sentences:
                        records.append(current_record)
                        current_record = ''
                        sentence_count = 0
                
                if sentence_count < max_sentences:
                    current_record += ' '  # Add space between paragraphs
            
            # If there is remaining text after the loop, add it as a record
            if current_record:
                records.append(current_record)
    
    return records


pdf_path = 'data/zurich_tax_info_2023.pdf'
max_sentences = 10  # Adjust the slice size as per your requirement

result = slice_pdf_into_records(pdf_path, max_sentences)

# print the length of result
print(len(result))

# Print the sliced records
for record in result:
    print(record)
    print('-------------------')
    #insert_requests(record)
    

print(db.isocodes.count_documents({}))


174
Kanton Zürich
Steueramt
Wegleitung zur 
Steuererklärung2022
Reichen Sie Ihre Steuererklärung online ein:
www.zh.ch/steuererklaerung
 
-------------------
A
Abonnementskosten (ZVV etc.)  15
AHV (Beiträge an die AHV)  18
AHV-Renten  11
Aktien  29
Alimente  12, 17
Anrechnung ausländischer 
Quellensteuern  30
Ausländische Arbeitnehmer  6
B
Bargeld  24
Baurechtszinsen  14
Behinderungsbedingte Kosten  18
Beihilfen  11
Beiträge Säule 3a  17
Berufsauslagen  15
Berufskleider  16
Betreuungskosten Kinder  19
Bundessteuertarife  41
Bussen  7
C
Checkliste zum Wertschriften -
verzeichnis  30
D
Darlehen  29
Dienstaltersgeschenke  10
E
Eigenmietwert  13
Einfamilienhaus / Eigenheim  13, 25
Eingetragene Partnerschaft  4
Einkommen (Bemessungsgrundlage)  5
Einschlag auf dem Eigenmietwert
– in Härtefällen  13
– Unternutzungsabzug  13
Erbengemeinschaften  7, 26
Erbschaft  7
Erbschaften  26
Erbvorbezug  7, 26
Ergänzungsleistungen  11
eSteuerauszug  27F
Fachliteratur  16
Fahrkilometer  15
Fahrkosten  15
F

In [13]:


def vector_search(vector_query, max_number_of_results=2):
  results = collection.aggregate([
    {
      '$search': {
        "cosmosSearch": {
          "vector": vector_query,
          "path": "vectorContent",
          "k": max_number_of_results
        },
      "returnStoredSource": True
      }
    }
  ])
  return results

# openAI request - ChatGPT 3.5 Turbo Model
def openai_request(prompt, model_engine='gpt-3.5-turbo'):
    completion = openai.ChatCompletion.create(model=model_engine, messages=prompt, temperature=0.2, max_tokens=500)
    return completion.choices[0].message.content


# define OpenAI Prompt for News Tweet
def create_tweet_prompt(user_question, result_json):
    instructions = f'You are a tool assistant that answers questions on tax: {result_json}. \
    Users can ask question in German and English. \
    The answer should be in the same language as the question.'
    task = f"{user_question}"
    
    # reminder to add some additional rules to the prompt
    
    prompt = [{"role": "system", "content": instructions }, 
              {"role": "user", "content": task }]
        # sample = 'Found tool SDNCN1616E11 :​
        #         cutting edge angle(KAPR1) 62.5 deg​
        #         lead angle(PSIR) 27.5 deg​'
    
    prompt = prompt 
    return prompt


# Step #4 Vector Search Tests

In [14]:


user_question = "Wann muss ich meine Steuererklärung einreichen?"

# generate embeddings for the question
user_question_embeddings = create_embeddings_with_openai(user_question)

# search for the question in the cosmos db
search_results = vector_search(user_question_embeddings, 1)
print(search_results)

# prepare the results for the openai prompt
result_json = []

# remove all empty values from the results json
search_results = [x for x in search_results if x]

# print each document in the result
for doc in search_results:
    print(doc.get('_id'), doc.get('textContent'), doc.get('vectorContent')[0:5])
    result_json.append(doc.get('textContent'))


# create the prompt
prompt = create_tweet_prompt(user_question, result_json)
display(prompt)

# generate the response
response = openai_request(prompt)

print(response)


Generating response from OpenAI...
1536
647130e45b0886ed706a98cd 4Wer hat im Kalenderjahr 2023 
eine Steuererklärung 2022 
 einzureichen?
Eine Steuererklärung 2022 haben im Kalenderjahr 2023 alle natürlichen Personen einzureichen, 
die am 31. Dezember 2022
• im Kanton Zürich Wohnsitz hatten oder
• im Kanton Zürich Liegenschaften oder Betriebsstätten (bzw. Geschäftsbetriebe)  besassen.
Zudem haben Steuerpflichtige mit Wohnsitz in einem anderen Kanton auch dann erst im 
 Kalenderjahr 2023 eine Steuererklärung 2022 einzureichen, wenn sie im Laufe des Kalender -
jahres 2022 ihre Steuerpflicht im Kanton Zürich durch Aufgabe einer Liegenschaft oder Be -
triebsstätte beendet haben.
Die Stellung der Partnerinnen oder Partner bei eingetragenen Partnerschaften entspricht seit 
dem 1.  Januar 2007 derjenigen von Ehegatten. Ausführungen unter dem Titel «Ehegatten» 
gelten auch für Partnerinnen oder Partner. Beim Ausfüllen der Steuererklärung sind die Anga-
ben derjenigen Person, deren Nachname alp

[{'role': 'system',
  'content': "You are a tool assistant that answers questions on tax: ['4Wer hat im Kalenderjahr 2023 \\neine\\xa0Steuererklärung 2022 \\n einzureichen?\\nEine Steuererklärung 2022 haben im Kalenderjahr 2023 alle natürlichen Personen einzureichen, \\ndie am 31. Dezember 2022\\n• im Kanton Zürich Wohnsitz hatten oder\\n• im Kanton Zürich Liegenschaften oder Betriebsstätten (bzw. Geschäftsbetriebe)  besassen.\\nZudem haben Steuerpflichtige mit Wohnsitz in einem anderen Kanton auch dann erst im \\n Kalenderjahr 2023 eine Steuererklärung 2022 einzureichen, wenn sie im Laufe des Kalender -\\njahres 2022 ihre Steuerpflicht im Kanton Zürich durch Aufgabe einer Liegenschaft oder Be -\\ntriebsstätte beendet haben.\\nDie Stellung der Partnerinnen oder Partner bei eingetragenen Partnerschaften entspricht seit \\ndem 1.  Januar 2007 derjenigen von Ehegatten. Ausführungen unter dem Titel «Ehegatten» \\ngelten auch für Partnerinnen oder Partner. Beim Ausfüllen der Steuererklärung

Das hängt von verschiedenen Faktoren ab, wie zum Beispiel Ihrem Wohnsitz oder dem Besitz von Liegenschaften oder Betriebsstätten im Kanton Zürich. Im Allgemeinen müssen alle natürlichen Personen, die am 31. Dezember 2022 im Kanton Zürich Wohnsitz hatten oder Liegenschaften oder Betriebsstätten besaßen, im Kalenderjahr 2023 eine Steuererklärung 2022 einreichen. Steuerpflichtige mit Wohnsitz in einem anderen Kanton müssen ebenfalls im Kalenderjahr 2023 eine Steuererklärung 2022 einreichen, wenn sie im Laufe des Jahres 2022 ihre Steuerpflicht im Kanton Zürich durch Aufgabe einer Liegenschaft oder Betriebsstätte beendet haben. Wenn Sie weitere Fragen haben, können Sie gerne präzisieren.


In [17]:
# display all documents in the collection
for doc in collection.find({}):
    print(doc.get('_id'), doc.get('textContent')[0:40], doc.get('vectorContent')[0:5])

In [7]:
# delete and recreate the index. This might only be necessary once.
collection.drop_indexes()

embedding_len = 1536
COLLECTION_NAME = 'isocodes'
db.command({
  'createIndexes': COLLECTION_NAME,
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "vectorContent": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 100,
        'similarity': 'COS',
        'dimensions': embedding_len
      }
    }
  ]
})

{'raw': {'defaultShard': {'numIndexesBefore': 1,
   'numIndexesAfter': 2,
   'createdCollectionAutomatically': False,
   'ok': 1}},
 'ok': 1}